In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-22 01:25:19--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-22 01:25:19 (13.8 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = "The course has already started, can I still get enrolled?"

In [9]:
index.fit(documents)

In [10]:
boost = {
    "question": 3.0,
    "section": 0.5
}

filters = {
    "course": "data-engineering-zoomcamp",
}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict=filters
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.\nYou can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.',
  'section': 'General course-related questions',
  'question': 'Course - Can I get support if I take the course in the self-paced mode?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materia

In [12]:
from openai import OpenAI

In [22]:
client = OpenAI()

In [23]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

In [25]:
response.choices[0].message.content

"It's typically possible to enroll in a course after it has started, but the specifics will depend on the policies of the institution or organization offering the course. Here’s what you should do:\n\n1. **Contact the Instructor or Administration**: Reach out to the course instructor or the administrative office as soon as possible. Explain your situation and express your interest in joining the course despite the late start.\n\n2. **Review the Enrollment Policies**: Check the institution’s policies regarding late enrollment. Some courses may have strict cutoff dates, while others might be more flexible.\n\n3. **Catch Up on Missed Content**: If you're allowed to enroll late, be prepared to catch up on any material you’ve missed. Ask for any resources, lecture notes, or recordings that could help you get up to speed.\n\n4. **Be Proactive**: Demonstrate your commitment to the course. This might mean dedicating extra time initially to ensure you don’t fall behind further.\n\nRemember, eac

In [38]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based in the provided CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering.
If the CONTEXT doesn't contain the answer, output NONE. Don't hallucinate.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [39]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

In [40]:
print(context)

section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I get support if I take the course in the self-paced mode?
answer:Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.
You can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.

section: General course-related questions
question:Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishe

In [43]:
prompt = prompt_template.format(question=q, context=context).strip()

In [44]:
print(prompt)

You are a course teaching assistant. Answer the QUESTION based in the provided CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering.
If the CONTEXT doesn't contain the answer, output NONE. Don't hallucinate.

QUESTION: The course has already started, can I still get enrolled?

CONTEXT: 
section: General course-related questions
question:Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question:Course - Can I get support if I take the course in the self-paced mode?
answer:Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already an

In [45]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [46]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### Making it modular

In [95]:
def search(query):
    boost = {
        "question": 3.0,
        "section": 0.5
    }
    
    filters = {
        "course": "data-engineering-zoomcamp",
    }
    
    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5,
        filter_dict=filters
    )

    return results

In [97]:
def build_prompt(query, search_results):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based in the provided CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [91]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [99]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [101]:
answer = rag("How do I run Kafka?")

In [102]:
print(answer)

To run Kafka, follow these steps based on the context provided:

1. **Java Kafka:**
   - Navigate to your project directory.
   - Use the following command to run your producer/consumer/kstreams/etc. in the terminal:
     ```
     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
     ```

2. **Python Kafka:**
   - Ensure you have a virtual environment set up:
     ```
     python -m venv env
     ```
   - Activate the virtual environment:
     - On MacOS and Linux:
       ```
       source env/bin/activate
       ```
     - On Windows:
       ```
       env\Scripts\activate
       ```
   - Install the required dependencies:
     ```
     pip install -r ../requirements.txt
     ```
   - To deactivate the virtual environment when done:
     ```
     deactivate
     ```

For issues like "NoBrokersAvailable," ensure your Kafka broker Docker container is running. Use:
```
docker ps
```
to check running containers, and
```
docker compose up -d
`

### Using Elasticsearch

In [103]:
from elasticsearch import Elasticsearch

In [104]:
es_client = Elasticsearch('http://localhost:9200')

In [107]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [109]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [110]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [113]:
query = "I just discovered the course, can I still join?"

In [122]:
def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [124]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [125]:
answer = rag(query)

In [126]:
print(answer)

Yes, you can still join the course even if it has already started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects.
